# TensorFlow Tutorial - 4. MLP ( Multi Layer Perceptron )

본 문서는 TensorFlow 를 사용하여 Deep Learning을 구현하기 위한 기초적인 실습 자료이다. 첫 번째 파트에서는 tensorflow에 대한 기본적인 설명과 deep learning 예제를 다루어보고, 두 번째 파트에서는 오픈소스를 활용한 Deep Reinforcement Learning 을 실습해보는 시간을 갖는다. 마지막으로는 TensorFlow로 구현되고 공개된 여러 오픈소스를 둘러본다.

The code and comments are written by Dong-Hyun Kwak <imcomking@gmail.com>
Upgraed to Tensorflow v1.1 by Buss NamJungGu <nowage@gmail.com> 

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.





 관련함수선언

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()

 정리되어 있는 input_data 가져오기

In [ ]:
# download the mnist data.
mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 

 데이터 형태 보기

In [ ]:
print(mnist.train.images.shape,mnist.train.labels.shape)
print(mnist.train.labels[1])

 데이터가 잘 들어 있는지 확인해보기

In [ ]:
for i in range(28):
    s=''
    for j in range(28):
        if mnist.train.images[1][i*28+j] >0.9 :
            s=s+'O'
        else:
            s=s+' '    
    print(s)            

 MLP 모델 정의

In [ ]:
# placeholder is used for feeding data.
x = tf.placeholder("float", shape=[None, 784], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target') # shape argument is optional, but this is useful to debug.


# all the variables are allocated in GPU memory
W1 = tf.Variable(tf.zeros([784, 256]), name = 'W1')   # create (784 * 256) matrix
b1 = tf.Variable(tf.zeros([256]), name = 'b1')        # create (1 * 256) vector
h1 = tf.sigmoid(tf.matmul(x, W1) + b1, name = 'h1')   # compute --> sigmoid(weighted summation)

# Repeat again
W2 = tf.Variable(tf.zeros([256, 10]), name = 'W2')     # create (256 * 10) matrix
b2 = tf.Variable(tf.zeros([10]), name = 'b2')          # create (1 * 10) vector
y = tf.nn.softmax(tf.matmul(h1, W2) + b2, name = 'y')  # compute classification --> softmax(weighted summation)


# define the Loss function
cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')


# define optimization algorithm
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)



correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
# correct_prediction is list of boolean which is the result of comparing(model prediction , data)


accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
# tf.cast() : changes true -> 1 / false -> 0
# tf.reduce_mean() : calculate the mean



# create summary of parameters
tf.summary.histogram('weights_1', W1)
tf.summary.histogram('weights_2', W2)
tf.summary.histogram('y', y)
tf.summary.scalar('cross_entropy', cross_entropy)
merged = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("/tmp/mlp")

 트레이닝

In [ ]:
# Create Session
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) # open a session which is a envrionment of computation graph.
sess.run(tf.global_variables_initializer())# initialize the variables

# training the MLP
for i in range(5001): # minibatch iteraction
    batch = mnist.train.next_batch(100) # minibatch size
    sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1]}) # placeholder's none length is replaced by i:i+100 indexes

    if i%500 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
        print ("step %d, training accuracy: %.3f"%(i, train_accuracy))



        # calculate the summary and write.
        summary = sess.run(merged, feed_dict={x:batch[0], y_target: batch[1]})
        summary_writer.add_summary(summary , i)

# for given x, y_target data set
print  ("test accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.test.images, y_target: mnist.test.labels}))
sess.close()